# This is  a research notebook to try to find correlation between hyperwave and volume 

In [1]:
!pip install --upgrade pip
!pip install quandl

Requirement already up-to-date: pip in /home/nbuser/anaconda3_501/lib/python3.6/site-packages (18.0)
    100% |████████████████████████████████| 51kB 4.4MB/s ta 0:00:011
  Running setup.py bdist_wheel for inflection ... done
  Stored in directory: /home/nbuser/.cache/pip/wheels/9f/5a/d3/6fc3bf6516d2a3eb7e18f9f28b472110b59325f3f258fe9211
Successfully built inflection


In [2]:
import time
from datetime import datetime, date, time, timedelta
import json
import requests
import os.path as path

from scipy.spatial import ConvexHull

import quandl
quandl.ApiConfig.api_key = "cEofBzyzyihN3fj62kp4"

import pandas as pd
import numpy as np

import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='davzucky', api_key='aZw7LRJOSDcPJyIk2G0U')
# This is to avoid warning when dividing by zero
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [3]:
path.abspath('.')

'/home/nbuser/library'

In [4]:
# mydata =  pd.DataFrame(quandl.get("FRED/GDP",returns="numpy", collapse="weekly",qopts = { 'columns': ['ticker', 'date', 'close', 'open', 'low', 'high'] },))
# mydata['diff'] = mydata['open'] - mydata['close']


In [5]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(mydata)

In [6]:
# f = mydata
# # display(f['open'])
# f = f.set_index('date')

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(weekly_data )

## This is the place where you setup the symbol and phases

In [7]:
data_3DSystemsCorp ={
    'name':'3D SYSTEMS CORP',
    'data_source':'quandl',
    'symbol':'DDD',
}


data_Valeant = {
    'name':'Baush health Company. Ex Valeant',
    'data_source':'quandl',
    'symbol':'VRX',
}

data_Amazon = {
    'name':'Amazon',
    'data_source':'investopedia',
    'symbol':'AMZN',
}

data_Square = {
    'name':'Square',
    'data_source':'investopedia',
    'symbol':'SQ',
}

data_Netflix= {
    'name':'Netflix',
    'data_source':'investopedia',
    'symbol':'NFLX',
}

data_Apple = {
    'name':'Apple',
    'data_source':'investopedia',
    'symbol':'AAPL',
}

data_Intuit = {
    'name':'Intuit',
    'data_source':'investopedia',
    'symbol':'INTU',
}

data_Gilead_Sciences = {
    'name':'Gilead Sciences',
    'data_source':'investopedia',
    'symbol':'GILD',
}

data_Abiomed = {
    'name':'Abiomed',
    'data_source':'investopedia',
    'symbol':'ABMD',
}

data_Alphabet = {
    'name':'Alphabet',
    'data_source':'investopedia',
    'symbol':'GOOGL',
}

data_BTCUSD = {
    'name':'BTCUSD',
    'data_source':'CryptoCompare',
    'symbol':'BTC-USD',
}

data_DowJones = {
    'name':'Dow Jones',
    'data_source':'investopedia',
    'symbol':'^DJI',
}

data_Chevron = {
    'name':'Chevron',
    'data_source':'investopedia',
    'symbol':'CVX',
}


data_DowJones_1920_1933 = {
    'name':'Dow Jones 1920-1933',
    'data_source':'LocalData',
    'symbol':'DowJones_1920_1933',
}

data_DowJones = {
    'name':'Dow Jones Industrial',
    'data_source':'LocalData',
    'symbol':'DowJones_Full',
}


data_10Y_Treasury_Note = {
    'name':'10y_usd_treasury_note',
    'data_source':'LocalData',
    'symbol':'10y_usd_treasury_note',
}


data_Caterpilar = {
    'name':'Caterpilar',
    'data_source':'investopedia',
    'symbol':'CAT',
}
data_DR_Horton = {
    'name':'D.R. Horton',
    'data_source':'investopedia',
    'symbol':'DHI',
}


In [8]:
company_setup = data_BTCUSD  
name = company_setup ['name'] 
data_source = company_setup ['data_source'] 
symbol = company_setup ['symbol']

# constant used for other computation
root_date = datetime(1800, 1, 6)

# Helper function 
This section contain helper function that are here to load and clean the raw data

In [9]:
def get_nb_weeks(row, base_date):
    return int((row["date"]-base_date).days/7)

def add_weekid_and_price_is_closing_up(df, base_date ):
    df['is_price_closing_up'] = df.close > df.close.shift()
    df['weekId'] = df.apply (lambda row: get_nb_weeks (row, base_date),axis=1)
    if "volume" in list(df.columns.values):
        df = df.drop("volume", axis=1)
    return df.sort_values(by="date")

class Investopedia_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._symbol = symbol
        self._timeframe = timeframe
    
    def _clean_data(self, df):
        df.loc[:,('date')] = pd.to_datetime(df.loc[:,('Date')])
        df = df.rename(columns={'Adj. Close':'close', 'Low':'low', 'Open':'open', 'High':'high','Volume':'volume'})
        df = df.set_index('date')
        df['date'] = df.index
        return df

    def _fetch_data(self): 
        url_symbol = "https://www.investopedia.com/markets/api/partial/historical/?Symbol={}&Type=Historical+Prices&Timeframe={}&StartDate=Jan+01%2C+1900".format(self._symbol, self._timeframe)
        df_list = pd.read_html(url_symbol, header=0, parse_dates=True)
        df_price = df_list[0].dropna()
        return df_price
    
    def get_dataframe(self):
        raw_data = self._fetch_data()
        return self._clean_data(raw_data)

class Quandl_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._symbol = symbol
        self._timeframe = timeframe

    def _aggregate_ticker_weekly(self, df):    
        open = df.open.resample('W-MON').last()
        close = df.close.resample('W-FRI').last().resample('W-MON').last()
        high = df.high.resample('W-MON').max()
        low = df.low.resample('W-MON').min()
        vol = df.volume.resample('W-MON').sum() 
        
        weekly_data = pd.concat([open, close, high, low, vol], axis=1)
        weekly_data ['date'] = weekly_data .index
        return weekly_data

    def _fetch_daily_data(self):
        daily_tickers = quandl.get_table('WIKI/PRICES', \
                                         ticker = [self._symbol], \
                                         qopts = { 'columns': ['ticker', 'date', 'close', 'open', 'low', 'high', 'volume'] }, \
                                         date = { 'gte': '1900-01-01'}, \
                                         paginate=True) #, 'lte': '2016-12-31' 
        daily_tickers = daily_tickers.set_index('date')
        daily_tickers['date'] = daily_tickers.index
        return daily_tickers
    
    def get_dataframe(self):
        daily_dataframe = self._fetch_daily_data()
        if( self._timeframe == 'daily'):
            return daily_dataframe.dropna()
        
        return self._aggregate_ticker_weekly(daily_dataframe).dropna()

class CryptoCompare_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._symbol = symbol
        self._timeframe = timeframe

    def _aggregate_ticker_weekly(self, df):    
        open = df.open.resample('W-MON').last()
        close = df.close.resample('W-SUN').last().resample('W-MON').last()
        high = df.high.resample('W-MON').max()
        low = df.low.resample('W-MON').min()
        vol = df.volume.resample('W-MON').sum() 
        
        weekly_data = pd.concat([open, close, high, low, vol], axis=1)
        weekly_data ['date'] = weekly_data .index
        return weekly_data

    def _fetch_daily_data(self):
        from_symbol, to_symbol  = self._symbol.split('-')
        url = "https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym={}&allData=true&aggregate=3&e=CCCAGG".format( \
                                                                                                                        from_symbol, \
                                                                                                                        to_symbol)

        r = requests.get(url)
        array = json.dumps(r.json())
        
        data = json.loads(array)
        daily_tickers = pd.DataFrame(data["Data"])
        daily_tickers['date'] = pd.to_datetime(daily_tickers['time'],unit='s')
        daily_tickers = daily_tickers.rename( columns={"volumeto": "volume"})
        daily_tickers = daily_tickers.set_index('date')
        daily_tickers['date'] = daily_tickers.index
        return daily_tickers
    
    def get_dataframe(self):
        daily_dataframe = self._fetch_daily_data()
        if( self._timeframe == 'daily'):
            return daily_dataframe.dropna()
        
        return self._aggregate_ticker_weekly(daily_dataframe).dropna()

class LocalData_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._file_name = "{}.csv".format(symbol)
        self._timeframe = timeframe

    
    def get_dataframe(self):
        file_path = path.join(path.abspath('.'), 'data', self._file_name)
        df = pd.read_csv(file_path, header=0, parse_dates=True)
        df = df.rename(columns={column: column.lower() for column in df.columns})
        df.loc[:,('date')] = pd.to_datetime(df.loc[:,('date')])
        df = df.set_index('date')
        df['date'] = df.index
        return df
        

# https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&allData=true&aggregate=3&e=CCCAGG
    
sources = {
    "investopedia": Investopedia_Loader,
    "quandl": Quandl_Loader,
    "CryptoCompare": CryptoCompare_Loader,
    "LocalData": LocalData_Loader
}


def get_historical_data(symbol, source, base_date, timeframe='weekly'):
    source_class = sources[source](symbol, timeframe)
    df_raw = source_class.get_dataframe()
    df_with_weekId = add_weekid_and_price_is_closing_up( df_raw, base_date )
    df_with_weekId = df_with_weekId.reset_index(drop=True)
    df_with_weekId = df_with_weekId.set_index('weekId')
    df_with_weekId['weekId'] = df_with_weekId.index
    return df_with_weekId

In [10]:
# url = "https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&allData=true&aggregate=3&e=CCCAGG"
# df = pd.read_json(url)
# df = pd.DataFrame(df.locations.values.tolist())['Data']
# display(df)

# import json
# import pandas as pd
# import requests
# r = requests.get(url)
# # print r.json() #
# array = json.dumps(r.json())
# # print(data["Data"])
# data = json.loads(array)
# df = pd.DataFrame(data["Data"])
# df['date'] = pd.to_datetime(df['time'],unit='s')
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(df)

## Function that allow to calculate the weekid from a start date

In [11]:


def get_weekId_max_price(df):
    return df.loc[ df.loc[:,('close')].argmax()]['weekId']

def get_weekId_min_price(df):
    return df.loc[ df.loc[:,('close')].argmin()]['weekId']


In [12]:
# def get_weekId_cartesian_product(df):
#     df_from = df.loc[:,('weekId',"close")].rename(index=str, columns={"weekId": "weekId_from", "close": "close_from"})
#     df_to = df.loc[:,('weekId',"close")].rename(index=str, columns={"weekId": "weekId_to", "close": "close_to"})
#     df_cartesian = df_from.assign(foo=1).merge(df_to.assign(foo=1)).drop('foo', 1)

#     return df_cartesian [(df_cartesian .weekId_from < df_cartesian .weekId_to)]    

# From here we start fetching the data
Above was only about setting up some basic function

In [13]:
# df_daily_price_raw = get_raw_historical_data(symbol, 'Daily')
df_weekly_price  = get_historical_data(symbol, data_source, root_date, 'Weekly')

# df_weekly_price = add_weekid_and_price_is_closing_up(df_weekly_price_raw)

max_price_weekId = get_weekId_max_price(df_weekly_price)

df_weekly_price_until_max = df_weekly_price [(df_weekly_price.weekId <= max_price_weekId)]
min_price_before_max_weekId = get_weekId_min_price(df_weekly_price_until_max)
df_weekly_price_until_max = df_weekly_price_until_max [(df_weekly_price_until_max.weekId >= min_price_before_max_weekId)]
# weekId_close_from_to = get_weekId_cartesian_product(df_weekly_price_until_max)

In [14]:
print(max_price_weekId)
print(min_price_before_max_weekId)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_weekly_price_until_max)

11372
10986


,open,close,high,low,date,is_price_closing_up,weekId
weekId,,,,,,,
10986,0.05050,0.05050,0.08181,0.05000,2010-07-26,False,10986
10987,0.06785,0.06000,0.06990,0.05600,2010-08-02,True,10987
10988,0.06230,0.07100,0.07350,0.05700,2010-08-09,True,10988
10989,0.06529,0.06529,0.07690,0.06000,2010-08-16,False,10989
10990,0.06640,0.06500,0.06790,0.06120,2010-08-23,False,10990
10991,0.06500,0.06497,0.06900,0.03211,2010-08-30,False,10991
10992,0.06160,0.06160,0.06490,0.05961,2010-09-06,False,10992
10993,0.06366,0.06199,0.17500,0.06014,2010-09-13,True,10993
10994,0.05900,0.06210,0.06340,0.05761,2010-09-20,True,10994


In [15]:
# print(max_price_weekId)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(df_weekly_price)
# df_weekly_price.ix[:df_weekly_price_until_max]

In [16]:
def get_line_slope_and_origine(df, x1_col_name = 'x1', \
                               y1_col_name = 'y1', \
                               x2_col_name = 'x2', \
                               y2_col_name = 'y2', \
                               m_col_name = 'm', \
                               b_col_name = 'b'):
    df[m_col_name] = (df[y1_col_name] - df[y2_col_name]) / (df[x1_col_name] - df[x2_col_name])
    df[b_col_name] = df[y1_col_name] - (df[x1_col_name] * df[m_col_name])
    return df

def get_normalize_column(df, column_name = 'close'):
    column_name_normalize = '{}_normalize'.format(column_name )
    max_value = df.loc[df[column_name ].idxmax()][column_name ]
    min_value = df.loc[df[column_name ].idxmin()][column_name ]

    df[column_name_normalize] = (df.loc[:,(column_name)] - min_value)/ (max_value - min_value ) 
    return df

def get_mean_square_error(y_true, y_pred):
#     y_square_diff = np.square(y_true-y_pred)
    y_square_diff = y_true - y_pred
    return np.sum(y_square_diff) / len(y_true)

def nb_is_lower(y_true, y_pred):
    lower_item = y_true[y_true <= y_pred]
    return len(lower_item )
    
def get_y(x, m, b):
    return x * m + b

def calculate_mean_square_error(row, df):
    y_pred = get_y(df['weekId'], row['m'], row['b'])
    return get_mean_square_error(df['close'], y_pred)

def nb_cut_price_low(row, df):
    y_pred = get_y(df['weekId'], row['m'], row['b'])
    return nb_is_lower(df['low'], y_pred)

# def normalize_column(df, column_name = 'close'):
    

def get_hull_tangent(df_input, x_column_name = 'weekId', y_column_name = 'close'):
    df_input = get_normalize_column(df_input, 'close')
    df_input= get_normalize_column(df_input, 'weekId')
    hull = ConvexHull(df_input[[x_column_name,  y_column_name]].dropna())
    
    hull_results = [ [min(pair[0], pair[1]), max(pair[0], pair[1])] for pair in hull.simplices]
    data_from_to = [{"x1": df_input['weekId'].iloc[pair[0]], \
        "x1_date": root_date + timedelta(weeks = df_input['weekId'].iloc[pair[0]].item()), \
        "x1_normalize": df_input['weekId_normalize'].iloc[pair[0]], \
        "y1": df_input['close'].iloc[pair[0]], \
        "y1_normalize": df_input['close_normalize'].iloc[pair[0]], \
        "x2": df_input['weekId'].iloc[pair[1]], \
        "x2_date": root_date + timedelta(weeks = df_input['weekId'].iloc[pair[1]].item()), \
        "x2_normalize": df_input['weekId_normalize'].iloc[pair[1]], \
        "y2": df_input['close'].iloc[pair[1]], \
        "y2_normalize": df_input['close_normalize'].iloc[pair[1]]} for pair in hull_results]
    df = pd.DataFrame(data_from_to)    
    df = get_line_slope_and_origine(df)
    df = get_line_slope_and_origine(df, \
                                x1_col_name = 'x1_normalize', \
                                y1_col_name = 'y1_normalize', \
                                x2_col_name = 'x2_normalize', \
                                y2_col_name = 'y2_normalize', \
                                m_col_name = 'm_normalize', \
                                b_col_name = 'b_normalize')
    df['angle'] = np.rad2deg(np.arctan2(df['m'], 1))
    df['angle_normalize'] = np.rad2deg(np.arctan2(df['m_normalize'], 1))
    df['weeks'] = np.abs(df['x1'] - df['x2'])
    df['mean_error'] = df.apply(lambda row: calculate_mean_square_error (row, df_input),axis=1)
    df['nb_is_lower'] = df.apply(lambda row: nb_cut_price_low(row, df_input),axis=1)
    df['ratio_error_cut'] = df['mean_error'] / df['nb_is_lower']
    df['ratio_slope_y1_normalize'] = df['y1_normalize']/df['m_normalize']
    df['ratio_slope_y2_normalize'] = df['y2_normalize']/df['m_normalize']
    return df


# def calculate_line_standard(df):
#     df = get_line_slope_and_origine(df)
#     df['angle'] = np.rad2deg(np.arctan2(df['m'], 1))
#     df['days'] = np.abs(df['x1'] - df['x2'])
#     df['mean_square_error'] = df.apply(lambda row: calculate_mean_square_error (row, df),axis=1)
#     df['nb_is_lower'] = df.apply(lambda row: nb_cut_price_low(row, df),axis=1)
#     df['ratio_error_cut'] = df['mean_square_error'] / df['nb_is_lower']
#     df['ratio_slope_y1_normalize'] = df['y1_normalize']/df['m']
#     df['ratio_slope_y2_normalize'] = df['y2_normalize']/df['m']
#     return df

# def calculate_line_normalize(df):
#     df = get_line_slope_and_origine(df, \
#                                     x1_col_name = 'x1_normalize', \
#                                     y1_col_name = 'y1_normalize', \
#                                     x2_col_name = 'x2_normalize', \
#                                     y2_col_name = 'y2_normalize')
    
#     df['angle'] = np.rad2deg(np.arctan2(df['m'], 1))
#     df['days'] = np.abs(df['x1'] - df['x2'])
#     df['mean_square_error'] = df.apply(lambda row: calculate_mean_square_error (row, df),axis=1)
#     df['nb_is_lower'] = df.apply(lambda row: nb_cut_price_low(row, df),axis=1)
#     df['ratio_error_cut'] = df['mean_square_error'] / df['nb_is_lower']
#     df['ratio_slope_y1_normalize'] = df['y1_normalize']/df['m']
#     df['ratio_slope_y2_normalize'] = df['y2_normalize']/df['m']
#     return df


# def get_hull_tangent_standard(df):
#     df = get_normalize_column(df, 'close')
#     df = get_normalize_column(df, 'weekId')
#     return calculate_line_standard(get_hull_tangent(df))


# def get_hull_tangent_normalize(df):
#     df = get_normalize_column(df, 'close')
#     df = get_normalize_column(df, 'weekId')
#     return calculate_line_normalize(get_hull_tangent(df, 'weekId_normalize', 'close_normalize'))
    
    
def get_upper_path(df):
    row = df.iloc[0]
    upper_path = [row['x1'], row['x2']]

    while(row['x2'] != max_price_weekId):
        row = df.loc[df['x1'] == row['x2']].head().iloc[0]
        upper_path.append(row['x2'])

    return upper_path

def delete_path(df, path):
    df_cleaned = df.copy(True)
    
    for x1, x2 in zip(path[:-1], path[1:]):
        df_cleaned = df_cleaned[(np.logical_not((df_cleaned.x1 == x1) & (df_cleaned.x2 == x2))) ]

    return df_cleaned.sort_values(['x1','y2'], ascending=[True, False])

def delete_upper_path(df):
    return delete_path(df, get_upper_path(df.sort_values(['x1','x2'], ascending=[True, False])))

def delete_shorter_path(df):
    df = df.sort_values(['x1','x2'], ascending=[True, False])
    path1 = get_upper_path(df)
    path2 = get_upper_path(df.iloc[1:])
    path = path1 if len(path1) < len(path2) else path2
    return delete_path(df, path)
    

In [17]:
# hull_results = [ [min(pair[0], pair[1]), max(pair[0], pair[1])] for pair in hull.simplices]
# data_from_to = [{"x1": df_weekly_price_until_max['weekId'].iloc[pair[0]], \
#    "y1": df_weekly_price_until_max['close'].iloc[pair[0]], \
#    "x2": df_weekly_price_until_max['weekId'].iloc[pair[1]], \
#    "y2": df_weekly_price_until_max['close'].iloc[pair[1]]} for pair in hull_results]
# df = pd.DataFrame(data_from_to)

In [18]:
def graph_hyperwave(df_data, df_hyperwave):

    x = df_data['weekId'].values
    y =  df_data['close_normalize'].values
    trace0 = go.Scatter(
        x = x,
        y = y,
        mode = 'lines',
        name = 'lines'
    )

    layout = {
    #     'xaxis': {
    #         'range': [0, 7]
    #     },
    #     'yaxis': {
    #         'range': [0, 2.5]
    #     },
        'shapes': [{
                'type': 'line',
                'x0': row[1]['x1'],
                'y0': row[1]['y1'],
                'x1': row[1]['x2'],
                'y1': row[1]['y2'],
                'line': {
                    'color': 'rgb(55, 128, 191)',
                    'width': 3,
                }
            } for row in df_hyperwave.iterrows()
            # Line Vertical
            ]
    }
    data = [trace0]

    # py.iplot(data, filename='line-mode')
    fig = {
        'data': data,
        'layout': layout,
    }

    return fig
#     py.iplot(fig, filename='shapes-lines')

In [58]:
df_hull_tangent = get_hull_tangent(df_weekly_price_until_max)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_hull_tangent.sort_values(['x1','x2'], ascending=[True, False]))
py.iplot(graph_hyperwave(df_weekly_price_until_max,df_hull_tangent), filename='hyperwave')
# df_hull_tangent = delete_upper_path(df_hull_tangent)

,x1,x1_date,x1_normalize,x2,x2_date,x2_normalize,y1,y1_normalize,y2,y2_normalize,m,b,m_normalize,b_normalize,angle,angle_normalize,weeks,mean_error,nb_is_lower,ratio_error_cut,ratio_slope_y1_normalize,ratio_slope_y2_normalize
0,10986,2010-07-26,0.000000,11372,2017-12-18,1.000000,0.0505,0.000000e+00,18972.3200,1.000000e+00,49.150957,-5.399724e+05,1.000000,0.000000,88.834450,45.000000,386,-8784.963940,387,-22.700165,0.000000,1.000000
18,10986,2010-07-26,0.000000,10996,2010-10-04,0.025907,0.0505,0.000000e+00,0.0614,5.745227e-07,0.001090,-1.192424e+01,0.000022,0.000000,0.062452,0.001271,10,700.960440,5,140.192088,0.000000,0.025907
22,10996,2010-10-04,0.025907,11007,2010-12-20,0.054404,0.0614,5.745227e-07,0.2401,9.993533e-06,0.016245,-1.785736e+02,0.000331,-0.000008,0.930714,0.018937,11,698.186992,12,58.182249,0.001738,0.030236
21,11007,2010-12-20,0.054404,11011,2011-01-17,0.064767,0.2401,9.993533e-06,0.3130,1.383598e-05,0.018225,-2.003625e+02,0.000371,-0.000010,1.044100,0.021245,4,697.846510,12,58.153876,0.026952,0.037314
20,11011,2011-01-17,0.064767,11022,2011-04-04,0.093264,0.3130,1.383598e-05,0.7790,3.839815e-05,0.042364,-4.661530e+02,0.000862,-0.000042,2.425807,0.049384,11,693.791219,8,86.723902,0.016053,0.044550
19,11022,2011-04-04,0.093264,11055,2011-11-21,0.178756,0.7790,3.839815e-05,2.2000,1.132969e-04,0.043061,-4.738350e+02,0.000876,-0.000043,2.465668,0.050196,33,693.681795,8,86.710224,0.043829,0.129321
9,11055,2011-11-21,0.178756,11082,2012-05-28,0.248705,2.2000,1.132969e-04,5.1400,2.682599e-04,0.108889,-1.201567e+03,0.002215,-0.000283,6.214390,0.126933,27,685.519088,5,137.103818,0.051141,0.121089
16,11082,2012-05-28,0.248705,11084,2012-06-11,0.253886,5.1400,2.682599e-04,5.5700,2.909246e-04,0.215000,-2.377490e+03,0.004374,-0.000820,12.133880,0.250626,2,675.226310,7,96.460901,0.061327,0.066508
17,11084,2012-06-11,0.253886,11106,2012-11-12,0.310881,5.5700,2.909246e-04,10.8700,5.702797e-04,0.240909,-2.664666e+03,0.004901,-0.000953,13.544973,0.280828,22,672.764947,10,67.276495,0.059355,0.116350
15,11106,2012-11-12,0.310881,11114,2013-01-07,0.331606,10.8700,5.702797e-04,13.5900,7.136468e-04,0.340000,-3.765170e+03,0.006917,-0.001580,18.778033,0.396335,8,665.531310,7,95.075901,0.082441,0.103166


In [57]:
df_hull_tangent.sort_values(['x1','x2'], ascending=[True, False])

,x1,x1_date,x1_normalize,x2,x2_date,x2_normalize,y1,y1_normalize,y2,y2_normalize,...,m_normalize,b_normalize,angle,angle_normalize,weeks,mean_error,nb_is_lower,ratio_error_cut,ratio_slope_y1_normalize,ratio_slope_y2_normalize
0,10986,2010-07-26,0.000000,11372,2017-12-18,1.000000,0.0505,0.000000e+00,18972.3200,1.000000e+00,...,1.000000,0.000000,88.834450,45.000000,386,-8784.963940,387,-22.700165,0.000000,1.000000
18,10986,2010-07-26,0.000000,10996,2010-10-04,0.025907,0.0505,0.000000e+00,0.0614,5.745227e-07,...,0.000022,0.000000,0.062452,0.001271,10,700.960440,5,140.192088,0.000000,0.025907
22,10996,2010-10-04,0.025907,11007,2010-12-20,0.054404,0.0614,5.745227e-07,0.2401,9.993533e-06,...,0.000331,-0.000008,0.930714,0.018937,11,698.186992,12,58.182249,0.001738,0.030236
21,11007,2010-12-20,0.054404,11011,2011-01-17,0.064767,0.2401,9.993533e-06,0.3130,1.383598e-05,...,0.000371,-0.000010,1.044100,0.021245,4,697.846510,12,58.153876,0.026952,0.037314
20,11011,2011-01-17,0.064767,11022,2011-04-04,0.093264,0.3130,1.383598e-05,0.7790,3.839815e-05,...,0.000862,-0.000042,2.425807,0.049384,11,693.791219,8,86.723902,0.016053,0.044550
19,11022,2011-04-04,0.093264,11055,2011-11-21,0.178756,0.7790,3.839815e-05,2.2000,1.132969e-04,...,0.000876,-0.000043,2.465668,0.050196,33,693.681795,8,86.710224,0.043829,0.129321
9,11055,2011-11-21,0.178756,11082,2012-05-28,0.248705,2.2000,1.132969e-04,5.1400,2.682599e-04,...,0.002215,-0.000283,6.214390,0.126933,27,685.519088,5,137.103818,0.051141,0.121089
16,11082,2012-05-28,0.248705,11084,2012-06-11,0.253886,5.1400,2.682599e-04,5.5700,2.909246e-04,...,0.004374,-0.000820,12.133880,0.250626,2,675.226310,7,96.460901,0.061327,0.066508
17,11084,2012-06-11,0.253886,11106,2012-11-12,0.310881,5.5700,2.909246e-04,10.8700,5.702797e-04,...,0.004901,-0.000953,13.544973,0.280828,22,672.764947,10,67.276495,0.059355,0.116350
15,11106,2012-11-12,0.310881,11114,2013-01-07,0.331606,10.8700,5.702797e-04,13.5900,7.136468e-04,...,0.006917,-0.001580,18.778033,0.396335,8,665.531310,7,95.075901,0.082441,0.103166


In [75]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(df_weekly_price_until_max)
# df =delete_upper_path(df_weekly_price_until_max)
# display(df)

class hyperwave:
    def __init__(self, \
                 min_m = 0.5, \
                 phase2_weeks_find_max = 156, \
                 phase_grow_factor = 2.2, \
                 phase4_min_weeks = 15, \
                 phase4_validation_previous_high = 1.4):
        self.phase2_weeks_find_max = phase2_weeks_find_max
        self.phase_grow_factor = phase_grow_factor
        self.min_m  = min_m  
        self.phase4_min_weeks= phase4_min_weeks 
        self.phase4_validation_previous_high = phase4_validation_previous_high 

    def get_hyperwave(self, df):
        # Step 1 - Get the raw Hull from max and min raw data
        df_min_to_max = self._borne_raw_data_between_max_to_min( df )
        df_hull = self._order_and_reset_index( self._delete_above_path( get_hull_tangent( df_min_to_max )))
        
        hw_phases_first_round = self._group_hyperwave_phase(df_hull)
        
        # Step 2 - Find max Price prior of start hyperwave
        first_phase_id = min(len(hw_phases_first_round), 3) * -1
        phase_2 = hw_phases_first_round [first_phase_id ]                
        min_week = self._get_phase_start_week( df_hull, phase_2 )
        max_price_weeks_before_start_week = self._get_max_price_week_before(df, min_week)
        
        hw_start_weekId = self._get_weekId_first_price_greater_than( df_min_to_max, \
                                                                    min_week, \
                                                                    max_price_weeks_before_start_week )
        
        # Step 3 - Get new Hull for the borned hyperwave raw data
        df_hyperwave_raw_data = df_min_to_max[(df_min_to_max.weekId >= hw_start_weekId )]

        df_hull_hyperwave = self._order_and_reset_index( \
                                        self._delete_above_path( get_hull_tangent( df_hyperwave_raw_data ))) 

        
        hw_phases_temp = self._group_hyperwave_phase(df_hull_hyperwave)
        
        hyperwave = {}
        phase_id = 1
        for phase in hw_phases_temp:
            phase_id = phase_id + 1
            df_phase =  df_hull_hyperwave.loc[df_hull_hyperwave.loc[phase].loc[:,('ratio_error_cut')].argmin()]
            hyperwave[ phase_id ] = df_phase[self._get_columns_not_normalize(df_phase)].to_dict()
        if len(hyperwave) >= 1:
            hyperwave[1] = self._get_phase1(hyperwave[2], max_price_weeks_before_start_week)
        return (df_hull_hyperwave, hw_phases_temp, hyperwave )

#     def df_result_row_to_dictionary(df_result):
    
    def _get_phase1(self, dic_phase2, price_break):
        dic_phase1 = dic_phase2.copy()
        dic_phase1['angle'] = 0
        dic_phase1['b'] = price_break
        dic_phase1['index'] = 0
        dic_phase1['m'] =  0
        dic_phase1['mean_error'] = 0
        dic_phase1['nb_is_lower'] = 0
        dic_phase1['ratio_error_cut'] = 0
        dic_phase1['weeks'] = 0
        return dic_phase1

    def _get_columns_not_normalize(self, df):
        return [c for c in df.axes[0] if "normalize" not in c]

    def _group_hyperwave_phase(self, df_result):
        filtered_hw = df_result[ (df_result.m_normalize > 0) ]
        
        current_phase_m = filtered_hw.iloc[0].m_normalize
        hw_phases_temp = []
        hw_current_phase = [filtered_hw.index[0]]
        
        for index, row in filtered_hw.loc[2:].iterrows():
            if row.m_normalize < current_phase_m * self.phase_grow_factor:
                hw_current_phase.append(index)
            else:
                hw_phases_temp.append(hw_current_phase)
                hw_current_phase = [index]
                current_phase_m = row.m_normalize
        
        hw_phases_temp.append(hw_current_phase)
        
        if len(hw_phases_temp) == 3:
            return hw_phases_temp
            
        for i in np.arange(len(hw_phases_temp)-1, 1, -1):
            phase = hw_phases_temp[i]
            if self._sum_group_weeks( filtered_hw, phase) < self.phase4_min_weeks:
                hw_phases_temp.remove( phase )
                hw_phases_temp[ i - 1 ].extend( phase )
        
        return hw_phases_temp

    def _order_and_reset_index(self, df):
        return df.sort_values(['x1','x2'], ascending=[True, False]) \
            .reset_index()


    def _sum_group_weeks(self, df, group):
        return df.loc[group].sum()['weeks']
    
    def _get_weekId_first_price_greater_than(self, df, min_week_id, max_price):
        df_week_greater_than = df[(df.weekId >= min_week_id)]

        df_val_price_greater_than_max = df_week_greater_than[(df_week_greater_than.close > max_price)]
        return df_val_price_greater_than_max.loc[ df_val_price_greater_than_max.loc[:,('weekId')].argmin()]['weekId']

    def _get_phase_start_week(self, df_result, phase_lines):
        return min(df_result.iloc[phase_lines]['x1'])
        
        
    def _delete_above_path(self, df):
# As we are using Hull to find the external phase of the graph. The positive mean_error as the way up
# whereas the negative are the way down
        return df[(df.mean_error >= 0)]

        
    def _delete_below_path(self, df):
# As we are using Hull to find the external phase of the graph. The positive mean_error as the way up
# whereas the negative are the way down
        return df[(df.mean_error < 0)]

    def _borne_raw_data_between_max_to_min(self, df):
#     Born the dataframe from with all the value before weekId of Max and from them find the min to born the other side
        max_price_weekId = self._get_weekId_max_price(df)
        df_until_max = df.loc[:max_price_weekId]
        min_price_weekId = self._get_weekId_min_price( df_until_max )    
        df_min_to_max = df_until_max.loc[min_price_weekId:]
        return df_min_to_max

    def _get_weekId_max_price(self, df):
        return df.loc[ df.loc[:,('close')].argmax(), 'weekId']

    def _get_weekId_min_price(self, df):
        return df.loc[ df.loc[:,('close')].argmin(), 'weekId']

    def _get_max_price(self, df, column_name = 'close'):
        return df.loc[ df.loc[:,(column_name)].argmax()][column_name]

    def _get_max_price_week_before(self, df, weekId):
        last_n_weeks_Items = df[(df.weekId <= weekId)].tail(self.phase2_weeks_find_max)    
        max_price = self._get_max_price( last_n_weeks_Items )
        return max_price


In [76]:
hw = hyperwave()

(df_result, phases, hyperwave) = hw.get_hyperwave(df_weekly_price)
# display(df_result)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_result)
    display(phases)
    display(hyperwave)
# display(df_result.loc[2:])
# hw._get_hw_start_week(df_result)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,index,x1,x1_date,x1_normalize,x2,x2_date,x2_normalize,y1,y1_normalize,y2,y2_normalize,m,b,m_normalize,b_normalize,angle,angle_normalize,weeks,mean_error,nb_is_lower,ratio_error_cut,ratio_slope_y1_normalize,ratio_slope_y2_normalize
0,1,11330,2017-02-27,0.000000,11334,2017-03-27,0.095238,1188.83,0.012186,969.44,0.000000,-54.847500,6.226110e+05,-0.127957,0.012186,-88.955478,-7.291774,4,4177.283314,4,1044.320828,-0.095238,-0.000000
1,4,11334,2017-03-27,0.095238,11337,2017-04-17,0.166667,969.44,0.000000,1176.80,0.011518,69.120000,-7.824366e+05,0.161254,-0.015358,89.171126,9.160330,3,2069.835814,6,344.972636,0.000000,0.071429
2,8,11337,2017-04-17,0.166667,11338,2017-04-24,0.190476,1176.80,0.011518,1264.31,0.016379,87.510000,-9.909241e+05,0.204157,-0.022508,89.345294,11.538785,1,1812.375814,9,201.375090,0.056418,0.080228
3,7,11338,2017-04-24,0.190476,11350,2017-07-17,0.476190,1264.31,0.016379,2320.23,0.075032,87.993333,-9.964041e+05,0.205285,-0.022723,89.348890,11.600792,12,1806.092481,9,200.676942,0.079787,0.365501
4,6,11350,2017-07-17,0.476190,11360,2017-09-25,0.714286,2320.23,0.075032,3932.83,0.164606,161.260000,-1.827981e+06,0.376213,-0.104117,89.644704,20.616959,10,1732.825814,8,216.603227,0.199440,0.437535
5,5,11360,2017-09-25,0.714286,11367,2017-11-13,0.880952,3932.83,0.164606,5878.13,0.272661,277.900000,-3.153011e+06,0.648330,-0.298486,89.793827,32.956535,7,2782.585814,6,463.764302,0.253893,0.420560
6,3,11367,2017-11-13,0.880952,11370,2017-12-04,0.952381,5878.13,0.272661,11246.21,0.570840,1789.360000,-2.033378e+07,4.174505,-3.404879,89.967980,76.528684,3,26965.945814,5,5393.189163,0.065316,0.136744
7,2,11370,2017-12-04,0.952381,11372,2017-12-18,1.000000,11246.21,0.570840,18972.32,1.000000,3863.055000,-4.391169e+07,9.012353,-8.012353,89.985168,83.668428,2,66366.150814,3,22122.050271,0.063340,0.110959


[[1, 2, 3], [4, 5], [6, 7]]

{1: {'angle': 0,
  'b': 1154.86,
  'index': 0,
  'm': 0,
  'mean_error': 0,
  'nb_is_lower': 0,
  'ratio_error_cut': 0,
  'weeks': 0,
  'x1': 11338,
  'x1_date': Timestamp('2017-04-24 00:00:00'),
  'x2': 11350,
  'x2_date': Timestamp('2017-07-17 00:00:00'),
  'y1': 1264.31,
  'y2': 2320.23},
 2: {'angle': 89.34889029752402,
  'b': -996404.1033333334,
  'index': 7,
  'm': 87.99333333333334,
  'mean_error': 1806.0924806201426,
  'nb_is_lower': 9,
  'ratio_error_cut': 200.67694229112695,
  'weeks': 12,
  'x1': 11338,
  'x1_date': Timestamp('2017-04-24 00:00:00'),
  'x2': 11350,
  'x2_date': Timestamp('2017-07-17 00:00:00'),
  'y1': 1264.31,
  'y2': 2320.23},
 3: {'angle': 89.64470392469312,
  'b': -1827980.77,
  'index': 6,
  'm': 161.26,
  'mean_error': 1732.8258139536117,
  'nb_is_lower': 8,
  'ratio_error_cut': 216.60322674420146,
  'weeks': 10,
  'x1': 11350,
  'x1_date': Timestamp('2017-07-17 00:00:00'),
  'x2': 11360,
  'x2_date': Timestamp('2017-09-25 00:00:00'),
  'y1': 2320.23,
 

In [35]:
df = df_weekly_price.loc[:11055]
display(df.index)
display(df)

df.index.idxmax()

Int64Index([10985, 10986, 10987, 10988, 10989, 10990, 10991, 10992, 10993,
            10994, 10995, 10996, 10997, 10998, 10999, 11000, 11001, 11002,
            11003, 11004, 11005, 11006, 11007, 11008, 11009, 11010, 11011,
            11012, 11013, 11014, 11015, 11016, 11017, 11018, 11019, 11020,
            11021, 11022, 11023, 11024, 11025, 11026, 11027, 11028, 11029,
            11030, 11031, 11032, 11033, 11034, 11035, 11036, 11037, 11038,
            11039, 11040, 11041, 11042, 11043, 11044, 11045, 11046, 11047,
            11048, 11049, 11050, 11051, 11052, 11053, 11054, 11055],
           dtype='int64', name='weekId')

,open,close,high,low,date,is_price_closing_up,weekId
weekId,,,,,,,
10985,0.04951,0.08080,0.09307,0.04951,2010-07-19,False,10985
10986,0.05050,0.05050,0.08181,0.05000,2010-07-26,False,10986
10987,0.06785,0.06000,0.06990,0.05600,2010-08-02,True,10987
10988,0.06230,0.07100,0.07350,0.05700,2010-08-09,True,10988
10989,0.06529,0.06529,0.07690,0.06000,2010-08-16,False,10989
10990,0.06640,0.06500,0.06790,0.06120,2010-08-23,False,10990
10991,0.06500,0.06497,0.06900,0.03211,2010-08-30,False,10991
10992,0.06160,0.06160,0.06490,0.05961,2010-09-06,False,10992
10993,0.06366,0.06199,0.17500,0.06014,2010-09-13,True,10993


AttributeError: 'Int64Index' object has no attribute 'idxmax'

In [24]:
df = df_weekly_price_until_max
max_weekId = 9689
nb_tail_items = 156

def get_max_price(df, column_name = 'close'):
    return df.loc[ df.loc[:,(column_name)].argmax()][column_name]
# weekId_max_price = get_weekId_max_price(last_n_weeks_Items) 
#     max_price_last_n_weeks = last_n_weeks_Items[(last_n_weeks_Items.weekId == weekId_max_price )].iloc[0]

last_n_weeks_Items = df[(df.weekId <= max_weekId)].tail(nb_tail_items)    
max_price = get_max_price(last_n_weeks_Items )

print( max_price)

df_week_greater_than = df[(df.weekId >= max_weekId)]

# df.loc[ df.loc[:,('close')].argmax()]['weekId']
df_val_price_greater_than_max = df_week_greater_than[(df_week_greater_than.close > max_price)]
weekId_start_hyperwave = df_val_price_greater_than_max.loc[ df_val_price_greater_than_max.loc[:,('weekId')].argmin()]['weekId']
df_week_greater_than = df[(df.weekId >= weekId_start_hyperwave )]

df = get_normalize_column(df_week_greater_than, 'close')
df = get_normalize_column(df, 'weekId')

df_hyperwave = get_hull_tangent(df,"weekId_normalize", "close_normalize").sort_values(['x1','y2'], ascending=[True, False])


df_hyperwave = delete_shorter_path(df_hyperwave)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_hyperwave)
py.iplot(graph_hyperwave(df_weekly_price_until_max,df_hyperwave), filename='hyperwave')
# py.iplot(fig, filename='shapes-lines')

ValueError: attempt to get argmax of an empty sequence

In [ ]:

# display(df_hyperwave.sort_values(['x1','y2'], ascending=[True, False]))
# print(max_price_weekId)
# print(get_upper_path(df_hyperwave))
df_low_path = delete_upper_path(df_hull_tangent)
display(df_hull_tangent)
display(df_low_path )
print(df_low_path.std()) 
print(df_low_path.mean()) #(axis=['m'])
print(df_low_path.median())

In [ ]:
# def get_line_slope(row):
#     return (row.close_from - row.close_to) / (row.weekId_from - row.weekId_to)


# weekId_close_from_to['slope'] = weekId_close_from_to.apply (lambda row: get_line_slope(row),axis=1)



# weekId_close_from_to

# # m = -0.110000
# # b = 1092.930000
# # m = 15.82
# # b = -177675.62
# def is_phase_below(m, b):
# #     m = row.m
# #     b = row.b
#     df = df_weekly_price
#     df_weekly_close = df[["weekId", "close"]]
#     df_weekly_close['ln_y'] = (df_weekly_close.weekId * m) + b
#     df_weekly_close['is_below'] = df_weekly_close.ln_y.le(df_weekly_close.close) | np.isclose(df_weekly_close['ln_y'], df_weekly_close['close'])
#     return df_weekly_close['is_below'].all()


# weekId_close_from_to['is_below'] = weekId_close_from_to.apply (lambda row: is_phase_below(row, df_weekly_price),axis=1)
# is_phase_below()

# weekId_close_from_to['r'] = weekId_close_from_to(weekId_close_from_to['m'], weekId_close_from_to['b'] )

# weekId_close_from_to
# import numpy as np
# import pandas as pd
# np.seterr(divide='ignore', invalid='ignore')



# df = df_weekly_price_until_max[:15].rename(columns={'weekId':'x', 'close': 'y'})


# data = {'x':[1,2,3,4],'y':[5,6,10,12]}
# df = pd.DataFrame(data)



# calculate m and b from y = mx = b
# df_m = (df['y'].values - df['y'].values[:, None]) / (df['x'].values - df['x'].values[:, None])
# df_b = df['y'].values - (df['x'].values * df_m)


# import itertools
# nb_partition = int((df['x'].values.shape[0] / 10)) + 1
# x_values_split = np.array_split(df['x'].values, nb_partition) 
# y_values_split = np.array_split(df['y'].values, nb_partition)              


# def get_y_values(x_value, m, b):
#     x_lenght = x_values.shape[0]
#     cube_shape = (x_lenght, 1, 1)
#     x_cube = np.reshape(x_values, cube_shape)
    
#     return (x_cube * m) + b 

# def get_is_line_above_close(x_values, y_values, m, b):
#     y_lenght = y_values.shape[0]
#     cube_shape = (y_lenght, 1, 1)
    
#     y_cube = np.reshape(y_values, cube_shape)
#     return get_y_values(x_values,m, b) < y_cube

    
# result = [get_y_vlowalues(x_values, df_m, df_b) for (x_values, y_values) in itertools.zip_longest(x_values_split, y_values_split )]
#     x_lenght = x_values.shape[0]
#     cube_shape = (x_lenght, 1, 1)
    
#     x_cube = np.reshape(x_values, cube_shape)
#     y_cube = np.reshape(y_values, cube_shape)
#     cube_y_value_calculated =  (x_cube * df_m) + df_b 

# x_values = x_values_split[0]
# y_values = y_values_split[0]              
# x_lenght = x_values.shape[0]
# cube_shape = (x_lenght, 1, 1)
https://www.google.com/search?q=hindsight+meaning&ie=utf-8&oe=utf-8&client=firefox-b-ab
# x_cube = np.reshape(x_values, cube_shape)
# y_cube = np.reshape(y_values, cube_shape)
